## Calculating Customer Lifetime Value

*note: calculations use equations from kissmetrics starbucks case study on customer churn*

* other cites used: http://www.dbmarketing.com/telecom/churnreduction.html

### Imports

In [1]:
import numpy as np
import pandas as pd
# import telco dataset from ibm
telco_df = pd.read_csv("https://s3.amazonaws.com/h2o-smalldata/TelcoChurn.csv",
                       converters={'TotalCharges':lambda x: pd.to_numeric(x, errors='coerce')})

### Set Variables

In [2]:

# average monthly bill per customer from the Telco datatsets Monthly Charges column
monthly_bill =  telco_df["MonthlyCharges"].mean()

# average customer lifespan (units are in years) using the telco tenure (which is measured in months)
average_customer_lifespan = telco_df['tenure'].mean()/12

# customer retention rate: assume churn rate is 1.5% on average for telecom companies
# churn_rate = 0.015
# churn_rate = 0.2
churn_rate = 0.02
retention = 1 - churn_rate

# profit margin per company: assume this is 50% on average for telecom companies
profit_margin = .5

# rate of discount: assume this is 10% assumming starbucks case study applies
rate_discount = .1

# average gross margin per customer lifespan (amount spent per customer over 2 year x profit margin)
gross_margin = profit_margin * (average_customer_lifespan * monthly_bill *12)

print 'monthly_bill:', monthly_bill
print 'average_customer_lifespan:',average_customer_lifespan
print 'retention:', retention
print 'profit_margin:', profit_margin
print 'rate_discount:', rate_discount
print 'gross_margin:', gross_margin

monthly_bill: 64.7616924606
average_customer_lifespan: 2.69759572152
retention: 0.98
profit_margin: 0.5
rate_discount: 0.1
gross_margin: 1048.205187


In [3]:
# create a class to get all these attributes
class customer_lifetime_value(object):
    
    def __init__(self,monthly_bill, average_customer_lifespan, 
                 churn_rate, profit_margin,rate_discount,gross_margin):
        self.monthly_bill = monthly_bill
        self.average_customer_lifespan = average_customer_lifespan
        self.churn_rate = churn_rate
        self.retention = 1 - self.churn_rate
        self.profit_margin = profit_margin
        self.rate_discount = rate_discount
        self.gross_margin = self.profit_margin * (self.average_customer_lifespan * self.monthly_bill *12)
        
    # simple lifetime value in $dollars
    def simple_lifetime(self): 
        self.simple_lifetime_value = self.monthly_bill * self.average_customer_lifespan * 12
        return self.simple_lifetime_value
    
    # traditional lifetime value in $dollars
    def traditional_lifetime(self):
        self.traditional_lifetime_value = self.gross_margin *((self.retention)/
                                             (1+self.rate_discount - self.retention))
        return self.traditional_lifetime_value
    
    # customer lifetime value in $dollars
    def lifetime(self):
        self.lifetime_value = self.average_customer_lifespan * 12 * \
                self.monthly_bill * self.profit_margin
        return self.lifetime_value 
    
    # average lifetime value in $dollars
    def average_lifetime(self):
        self.average_lifetime_value = np.mean([self.simple_lifetime()
                                         ,self.traditional_lifetime(),
                                         self.lifetime()])

        return self.average_lifetime_value   

In [4]:
churn_rate = 0.1
retention = 1 - churn_rate 
telco_churn_1 = customer_lifetime_value(monthly_bill, average_customer_lifespan, 
                 churn_rate, profit_margin, rate_discount,
                           gross_margin)

In [6]:
print 'simple_lifetime customer value:', telco_churn_1.simple_lifetime()
print 'traditional_lifetime customer value:',telco_churn_1.traditional_lifetime()
print 'lifetime customer value:', telco_churn_1.lifetime()
# average_lifetime is the average of the three different ways to calculate customer lifetime value
print 'average_lifetime customer value:', telco_churn_1.average_lifetime()


simple_lifetime customer value: 2096.410374
traditional_lifetime customer value: 4716.9233415
lifetime customer value: 1048.205187
average_lifetime customer value: 2620.5129675


In [7]:
# change the churn rate
churn_rate = .2
retention = 1 - churn_rate 
telco_churn_2 = customer_lifetime_value(monthly_bill, average_customer_lifespan, 
                 churn_rate, profit_margin, rate_discount,
                           gross_margin)

print telco_churn_2.simple_lifetime()
print telco_churn_2.traditional_lifetime()
print telco_churn_2.lifetime()
print telco_churn_2.average_lifetime()

2096.410374
2795.213832
1048.205187
1979.943131


In [78]:
# if you have 1 million subscribers, with a churn rate of 2%, and an average monthly revenue per user of $100 dollars
num_subscribers = 1.0e6
monthly_churn = .0121    
new_monthly_churn = .0111
monthly_rev_per_user = 50.


def revenue_saved(num_subscribers, monthly_churn, new_monthly_churn, monthly_rev_per_user):
    """returns the amount of revenue saved by decreasing/increasing churn rates. input: num_subscribers
    is the number of subscribers, monthly_churn is the current monthly churn, new_monthly_churn is
    the new monthly churn you would like, and monthly_rev_per_user is the amount of monthly revenue acquired
    from each subscriber"""
    revenue_1 = num_subscribers * monthly_churn * monthly_rev_per_user * 12
    revenue_2 = num_subscribers * new_monthly_churn * monthly_rev_per_user * 12
    revenue_final = revenue_1 - revenue_2 
    return revenue_final
    
better_churn = revenue_saved(num_subscribers, monthly_churn, new_monthly_churn, monthly_rev_per_user)

print "revenue_saved", better_churn
print "gross profits", monthly_rev_per_user* num_subscribers

print num_subscribers * new_monthly_churn * monthly_rev_per_user * 12
print num_subscribers * monthly_churn * monthly_rev_per_user * 12


revenue_saved 600000.0
gross profits 50000000.0
6660000.0
7260000.0


In [23]:
import math
# http://sixteenventures.com/saas-churn-rate (cite this if you use their txt)
def annual_churn_rate(churn_rate):
    """input your montly churn rate to calculate your annual churn rate"""
    retention_rate_per_year = (1-churn_rate)**12
    c_rate = 1 - retention_rate_per_year 
    return c_rate
    
churn_rate = .02

churn_percent = annual_churn_rate(churn_rate)

print "at an annual churn rate of {0} you need re-acquire {1} customers for every 100 customer.".format("%.2f" % churn_percent, \
                                                                             "%.2f" % (churn_percent*100))
print "this means to grow by 1 customer, you need to acquire {0}".format("%.2f" % (churn_percent*100))

at an annual churn rate of 0.22 you need re-acquire 21.53 customers for every 100 customer.
this means to grow by 1 customer, you need to acquire 21.53
